# 📊 Exploratory Data Analysis
## Agentic AI — Demand Blocking & Smart Rebooking

This notebook explores all 12 database tables to understand data distributions,
correlations, patterns, and anomalies before model training.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils.db_utils import get_sqlalchemy_engine, read_all_tables, test_connection

sns.set_theme(style='whitegrid', palette='viridis')
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

%matplotlib inline

## 1. Connect & Load All Tables

In [ ]:
engine = get_sqlalchemy_engine()
assert test_connection(engine), 'Database connection failed!'

data = read_all_tables(engine)

# Quick summary
summary = pd.DataFrame([
    {'Table': k, 'Rows': len(v), 'Columns': len(v.columns), 'Missing %': round(v.isnull().mean().mean()*100, 2)}
    for k, v in data.items()
])
summary

## 2. Property Master Analysis

In [ ]:
pm = data['property_master']
print(f'Properties: {len(pm)}')
print(f'Cities: {pm["city"].nunique()}')
display(pm.describe())

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# City tier distribution
pm['city_tier'].value_counts().plot.bar(ax=axes[0,0], color=['#3b82f6','#60a5fa','#93c5fd'])
axes[0,0].set_title('Properties by City Tier')

# Star rating distribution
pm['star_rating'].value_counts().sort_index().plot.bar(ax=axes[0,1], color='#f59e0b')
axes[0,1].set_title('Properties by Star Rating')

# Base ADR distribution
pm['base_adr_inr'].hist(ax=axes[1,0], bins=30, color='#10b981', edgecolor='white')
axes[1,0].set_title('Base ADR Distribution (INR)')

# Popularity index
pm['popularity_index'].hist(ax=axes[1,1], bins=30, color='#8b5cf6', edgecolor='white')
axes[1,1].set_title('Popularity Index Distribution')

plt.tight_layout()
plt.show()

## 3. City Demand Signals

In [ ]:
cds = data['city_demand_signals'].copy()
cds['date'] = pd.to_datetime(cds['date'])
print(f'Date range: {cds["date"].min()} to {cds["date"].max()}')
print(f'Cities: {cds["city"].nunique()}')

# Demand multiplier trends by city
fig = px.line(cds, x='date', y='city_demand_multiplier', color='city',
              title='City Demand Multiplier Over Time')
fig.add_hline(y=1.3, line_dash='dash', line_color='red', annotation_text='Spike Threshold')
fig.update_layout(height=500)
fig.show()

In [ ]:
# Seasonality and weekend patterns
fig = make_subplots(rows=1, cols=2, subplot_titles=('Seasonality', 'Weekend Multiplier'))
for city in cds['city'].unique()[:5]:
    city_data = cds[cds['city'] == city]
    fig.add_trace(go.Scatter(x=city_data['date'], y=city_data['seasonality'],
                             mode='lines', name=city), row=1, col=1)
    fig.add_trace(go.Scatter(x=city_data['date'], y=city_data['weekend_multiplier'],
                             mode='lines', name=city, showlegend=False), row=1, col=2)
fig.update_layout(height=400)
fig.show()

## 4. Property Daily — Occupancy & Sell-out Analysis

In [ ]:
pd_df = data['property_daily'].copy()
pd_df['date'] = pd.to_datetime(pd_df['date'])
pd_df['occupancy_rate'] = pd_df['rooms_sold'] / pd_df['base_inventory_rooms'].clip(lower=1)

print(f'Rows: {len(pd_df):,}')
print(f'Properties: {pd_df["property_id"].nunique()}')
print(f'Date range: {pd_df["date"].min()} to {pd_df["date"].max()}')
print(f'Sell-out rate: {pd_df["sold_out_flag"].mean():.1%}')

# Occupancy distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
pd_df['occupancy_rate'].hist(ax=axes[0], bins=50, color='#3b82f6', edgecolor='white')
axes[0].set_title('Occupancy Rate Distribution')
axes[0].axvline(pd_df['occupancy_rate'].mean(), color='red', linestyle='--', label=f'Mean: {pd_df["occupancy_rate"].mean():.2f}')
axes[0].legend()

# Booking requests
pd_df['booking_requests'].hist(ax=axes[1], bins=50, color='#f59e0b', edgecolor='white')
axes[1].set_title('Booking Requests Distribution')

# Sold-out by day of week
pd_df['day_of_week'] = pd_df['date'].dt.dayofweek
dow_sellout = pd_df.groupby('day_of_week')['sold_out_flag'].mean()
dow_sellout.plot.bar(ax=axes[2], color='#e74c3c')
axes[2].set_title('Sell-out Rate by Day of Week')
axes[2].set_xticklabels(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])

plt.tight_layout()
plt.show()

In [ ]:
# Sell-out frequency per property
sellout_freq = pd_df.groupby('property_id')['sold_out_flag'].mean().sort_values(ascending=False)
fig = px.bar(x=sellout_freq.head(30).index, y=sellout_freq.head(30).values,
             title='Top 30 Properties by Sell-out Frequency',
             labels={'x': 'Property', 'y': 'Sell-out Rate'})
fig.update_layout(height=400)
fig.show()

## 5. Supplier Reliability

In [ ]:
sup = data['supplier_reliability']
display(sup)

fig = px.bar(sup, x='supplier_name',
             y=['booking_failure_rate', 'supplier_cancellation_rate', 'dispute_rate'],
             barmode='group', title='Supplier Risk Metrics',
             color_discrete_sequence=['#e74c3c', '#f39c12', '#9b59b6'])
fig.update_layout(height=400)
fig.show()

## 6. Rate Snapshots & Price Analysis

In [ ]:
rates = data['rate_snapshots'].copy()
rates['snapshot_date'] = pd.to_datetime(rates['snapshot_date'])
print(f'Rate snapshots: {len(rates):,}')

# Price distribution by room type
fig = px.box(rates, x='standard_room_type', y='net_rate_inr', color='standard_room_type',
             title='Rate Distribution by Room Type',
             color_discrete_sequence=['#3b82f6', '#10b981', '#f59e0b'])
fig.update_layout(height=400)
fig.show()

In [ ]:
# Price trends over time by supplier
rate_trend = rates.groupby(['snapshot_date', 'supplier_id'])['net_rate_inr'].mean().reset_index()
fig = px.line(rate_trend, x='snapshot_date', y='net_rate_inr', color='supplier_id',
              title='Average Net Rate Over Time by Supplier')
fig.update_layout(height=400)
fig.show()

## 7. Confirmed Bookings Analysis

In [ ]:
bk = data['confirmed_bookings'].copy()
bk['booked_date'] = pd.to_datetime(bk['booked_date'])
bk['stay_checkin_date'] = pd.to_datetime(bk['stay_checkin_date'])
bk['lead_time'] = (bk['stay_checkin_date'] - bk['booked_date']).dt.days

print(f'Bookings: {len(bk):,}')
print(f'Status distribution:')
print(bk['booking_status'].value_counts())

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Lead time distribution
bk['lead_time'].clip(upper=90).hist(ax=axes[0,0], bins=40, color='#3b82f6', edgecolor='white')
axes[0,0].set_title('Booking Lead Time (days)')

# Markup distribution
bk['markup_pct'].hist(ax=axes[0,1], bins=40, color='#10b981', edgecolor='white')
axes[0,1].set_title('Markup % Distribution')

# Bookings by room type
bk['standard_room_type'].value_counts().plot.bar(ax=axes[1,0], color='#f59e0b')
axes[1,0].set_title('Bookings by Room Type')

# Cancellation type
bk['cancellation_type'].value_counts().plot.pie(ax=axes[1,1], autopct='%.1f%%', colors=['#27ae60','#f39c12','#e74c3c'])
axes[1,1].set_title('Cancellation Type Split')

plt.tight_layout()
plt.show()

## 8. Demand Blocking & Rebooking History

In [ ]:
blocks = data['demand_block_actions'].copy()
blocks['week_start'] = pd.to_datetime(blocks['week_start'])

rebook = data['rebooking_evaluations'].copy()
rebook['eval_week'] = pd.to_datetime(rebook['eval_week'])

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Weekly Rooms Blocked', 'Weekly Rebooking Decisions'))

# Blocking trend
wk_blocks = blocks.groupby('week_start')['rooms_blocked_per_night'].sum().reset_index()
fig.add_trace(go.Bar(x=wk_blocks['week_start'], y=wk_blocks['rooms_blocked_per_night'],
                     marker_color='#3b82f6', name='Rooms'), row=1, col=1)

# Rebooking trend
wk_rebook = rebook.groupby(['eval_week', 'decision']).size().reset_index(name='count')
for dec, color in [('Rebook', '#27ae60'), ('Skip', '#e74c3c')]:
    subset = wk_rebook[wk_rebook['decision'] == dec]
    fig.add_trace(go.Bar(x=subset['eval_week'], y=subset['count'],
                         marker_color=color, name=dec), row=1, col=2)

fig.update_layout(height=400, barmode='stack')
fig.show()

## 9. Weekly KPI Summary

In [ ]:
kpi = data['weekly_kpi_summary'].copy()
kpi['week_start'] = pd.to_datetime(kpi['week_start'])
kpi = kpi.sort_values('week_start')

display(kpi.tail(10))

fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('Properties Blocked', 'Expected Uplift (INR)',
                                    'Rebook Count', 'Margin Recovered (INR)'))
fig.add_trace(go.Scatter(x=kpi['week_start'], y=kpi['properties_blocked'],
                         mode='lines+markers', line=dict(color='#3b82f6')), row=1, col=1)
fig.add_trace(go.Bar(x=kpi['week_start'], y=kpi['expected_uplift_inr'],
                     marker_color='#10b981'), row=1, col=2)
fig.add_trace(go.Scatter(x=kpi['week_start'], y=kpi['rebook_count'],
                         mode='lines+markers', line=dict(color='#f59e0b')), row=2, col=1)
fig.add_trace(go.Bar(x=kpi['week_start'], y=kpi['margin_recovered_inr'],
                     marker_color='#8b5cf6'), row=2, col=2)
fig.update_layout(height=500, showlegend=False)
fig.show()

## 10. Correlation Analysis

In [ ]:
# Key correlations in Property_Daily
pd_df = data['property_daily'].copy()
numeric_cols = pd_df.select_dtypes(include=[np.number]).columns
corr = pd_df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, linewidths=0.5)
plt.title('Property Daily — Correlation Matrix')
plt.tight_layout()
plt.show()

## 11. Events Impact Analysis

In [ ]:
events = data['events_calendar'].copy()
events['start_date'] = pd.to_datetime(events['start_date'])

# Event type distribution
fig = px.sunburst(events, path=['city', 'event_type'], values='demand_intensity',
                  title='Events by City and Type (sized by demand intensity)',
                  color='demand_intensity', color_continuous_scale='YlOrRd')
fig.update_layout(height=500)
fig.show()

## 12. Room Mapping & Equivalence

In [ ]:
rm = data['room_mapping']
print(f'Room mappings: {len(rm):,}')
print(f'Avg equivalence score: {rm["equivalence_score"].mean():.3f}')

fig = px.histogram(rm, x='equivalence_score', color='standard_room_type',
                   title='Room Equivalence Score Distribution',
                   barmode='overlay', opacity=0.7,
                   color_discrete_sequence=['#3b82f6', '#10b981', '#f59e0b'])
fig.update_layout(height=350)
fig.show()

---
## Key Findings Summary

After running all cells above, summarize:
1. Which cities have highest demand volatility?
2. Which properties sell out most frequently?
3. Which suppliers are most/least reliable?
4. What is the typical booking lead time?
5. How do events correlate with demand spikes?
6. What is the price volatility across room types?